# Momentum method

> $\nabla f = 2A^T(Ax - b)$<br>
> $x_{k+1} = x_k - s \cdot z_k$<br>
> $z_{k+1} = \nabla f_{k+1} + \beta \cdot z_k$

$
\begin{bmatrix}
1 & 0 \\
-S & 1
\end{bmatrix}
\begin{bmatrix}
x_{k+1} \\ z_{k+1}
\end{bmatrix}
=
\begin{bmatrix}
1 & -s \\
0 & \beta
\end{bmatrix}
\begin{bmatrix}
x_k \\ z_k
\end{bmatrix}
$

With assumption that $x_k$ and $z_k$ is tracking S's eigenvector, $s_{optimize}$ and $\beta_{optimize}$ will be denoted as: <br>
$s_{optimize} = ({2 \over \sqrt{M} + \sqrt{m}})^2$<br>
$\beta_{optimize} = ({\sqrt{M} - \sqrt{m} \over \sqrt{M} + \sqrt{m}})^2$<br>
where $M, m$ is upperbound and downerbound of eigenvalues of S.

I need to calculate $s_{optimize}$ and $\beta_{optimize}$ for our object function __$f(x) = \begin{Vmatrix}Ax - b\end{Vmatrix}^2$__.

In [ ]:
import numpy as np

In [ ]:
## QR decomposition
def factorize(A):
    width = A.shape[1]

    Q = np.zeros_like(A)
    R = np.zeros((width,width))

    ## first step
    norm_q = np.linalg.norm(A[:,0])
    Q[:,0] = A[:,0] / norm_q
    R[0,0] = norm_q

    ## gram-schmidts algorithm
    for i in range(1,width):
        Q[:,i] = A[:,i]
        
        for j in range(i):
            Q[:,i] -= np.dot(Q[:,j], A[:,i]) * Q[:,j]
            R[j,i] = np.dot(Q[:,j],A[:,i])

        norm_q = np.linalg.norm(Q[:,i])
        Q[:,i] = Q[:,i] / norm_q
        R[i,i] = norm_q

        ## linearly dependence
        if norm_q == 0:
            print("it has linearly dependent columns")
            break

    return Q, R

In [ ]:
# your code here
it_is_all_real = 0

for i in range(1000):
    A = np.random.rand(100,10)
    A_gram = np.dot(A.T,A)

    w, v = np.linalg.eig(A_gram)

    w_r = np.real(w)
    w_i = w - w_r

    is_it_all_real = np.linalg.norm(w_i)

    if (is_it_all_real < 1e-12):
        it_is_all_real += 1
            
print(f"{it_is_all_real} matrices are has all real eigenvalues")
print(f"The portion of all is {it_is_all_real / 10} %")

1000 matrices are has all real eigenvalues
The portion of all is 100.0 %


In [1]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 31.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 78 kB 8.3 MB/s 
     |████████████████████████████████| 70 kB 8.7 MB/s 
  Created wheel for pycuda: filename=pycuda-2022.1-cp37-cp37m-linux_x86_64.whl size=629484 sha256=d6eb77e308eaa95ccb92ee756a1d3be691df7c3b1581ed3d9cc2bbd1c8f4ce41
  Stored in directory: /root/.cache/pip/wheels/17/53/c9/caa05618e686df51f017d8a9923f38d915ce31df67ab6628e6
  Created wheel for pytools: filename=pytools-2022.1.12-py2.py3-none-any.whl size=65034 sha256=82fd0c5fe2ec8dc205babc8029c740b5f782572c692848d743e01f68628a0a00
  Stored in directory: /root/.cache/pip/wheels/37/5e/9e/76d7430e116b7cab0016fbabb26b896daae1946a3f7dea9915
Successfully built pycuda pytools


In [2]:
import pycuda.autoinit
import pycuda.driver as drv
from pycuda import gpuarray
from pycuda.compiler import SourceModule
import numpy as np
import matplotlib.pyplot as plt
from time import time
import math

```python
import pycuda.autoinit

calculate_ker = SourceModule(
    """
    #define _X (threadIdx.x + blockIdx.x * blockDim.x * blockDim.y)

    #define _WIDTH (blockDim.x)

    #define _INDEX1(x,y) (x * _WIDTH + y)

    // A: matrix, b: vector, out: vector
    __global__ void mat_vec_ker(float *out, float *A, float *b, float *theta)
    {
        int x = _X;

        for (int j = 0; j < _WIDTH; j++)
        {
            // out need to be initialized
            out[x] += A[_INDEX1(x,j)] * theta[j];
        }

        __syncthreads();
        out[x] -= b[x];
        __syncthreads();
    }
    """
)

gradient_ker = SourceModule(
    """
    #define _X (threadIdx.x)
    #define _B (blockIdx.x)
    #define _G (gridDim.x)

    #define _WIDTH (blockDim.x)

    // grad_jerk: [BD,n], n = gridDim.x
    __global__ void gradient_ker(float *grad_jerk, float *out, float *A, int width)
    {
        int x = _X;
        int index_g = x * _G + _B;
        int index_a;
        int index_o;

        for (int k = 0; k < _WIDTH; k++)
        {
            index_a = x + k * _WIDTH + _B * _WIDTH * _WIDTH;
            index_o = k + _B * _WIDTH;

            grad_jerk[index_g] += A[index_a] * out[index_o];
        }

        __syncthreads();
    }

    __global__ void finish_ker(float *grad, float *grad_jerk)
    {
        int x = _X;

        for (int k = 0; k < _G; k++)
        {
            int index = x * _G + k;
            grad[x] += grad_jerk[index];
        }
        __syncthreads();
    }
    """
)

update_ker = SourceModule(
    """
    #define _X (threadIdx.x + blockIdx.x * blockDim.x * blockDim.y)

    __global__ void update_ker(float *theta_new, float *theta, float lr, float *grad)
    {
        int x = _X;

        theta_new[x] = theta[x] - grad[x] * lr;

        __syncthreads();
    }
    """
)

multiply = calculate_ker.get_function("mat_vec_ker")
gradient = gradient_ker.get_function("gradient_ker")
finish = gradient_ker.get_function("finish_ker")
update = update_ker.get_function("update_ker")
```

In [ ]:
def optimal_block_size(n):
    
    thread_per_block = int(math.sqrt(n / 2))

    block_per_grid = int(n / thread_per_block) + 1


    return thread_per_block, block_per_grid



## block=(thread_per_block,1,1), grid=(block_per_grid,1,1)
first_ker = SourceModule(
    """
    #define x (threadIdx.x + blockIdx.x * blockDim.x)

    __global__ void first_ker(float* out, float* A, float* theta, int length, int width) {
        
        if (x < length) {
            for (int j = 0; j < width;, j++) {
                int index = x * width + j;

                out[x] += A[index] * theta[j];
            }
        }
    }
    """
)



## block=(thread_per_block,1,1), grid=(block_per_grid,1,1)
second_ker = SourceModule(
    """
    #define x (threadIdx.x + blockIdx.x * blockDim.x)

    __global__ void second_ker(float* out, float* b, int length) {

        if (x < length) {
            out[x] = out[x] - b[x];
        }
    }
    """
)



## block=(block_per_grid,1,1), grid=(width,1,1)
third_ker = SourceModule(
    """
    __global__ void third_ker(float* grad, float* A, float* out, int thread_per_block, int block_per_grid, int width) {

        __shared__ float* grad_jerk[gridDim.x];

        for (int i = 0; i < thread_per_block; i++) {
            int index1 = threadIdx.x * block_per_grid + i;
            int index2 = index1 * gridDim.x + blockIdx.x;
             
            grad_jerk[threadIdx.x] += A[index2] * out[index1];
        }

        if (threadIdx.x == 0) {
            for (int i = 0; i < gridDim.x; i++) {
                grad[x] += grad_jerk[i];
            }
        }
    }
    """
)



## block=(width,1,1), grid=(1,1,1)
gradient_method_ker = SourceModule(
    """
    #define x (threadIdx.x)

    __global__ void gradient_method (float* n_theta, float* theta, float* grad, float learning_rate) {
        n_theta[x] = theta[x] - learning_rate * grad[x];
    }
    """
)



## block=(width,1,1), grid=(1,1,1)
momentum_method_ker = SourceModule(
    """
    #define x (threadIdx.x)

    __global__ void momentum_method (float* n_theta, float* theta, float* grad, float* n_momentum, float* momentum, float s, float beta) {
        n_theta[x] = theta[x] - s * momentum[x];
        n_momentum[x] = grad[x] + beta * momentum[x];
    }
    """
)